In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import random
import numpy as np
from torch.nn import Module
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union
from torch import Tensor

In [ ]:
class CustomTrainDataset1(Dataset,Module):
    def __init__(self, folder_path, index_array, transform=None):
        self.folder_path = folder_path
        self.image_files=[os.path.join(folder_path,file) for file in os.listdir(folder_path)]
        self.image_paths=random.sample(self.image_files,7000)
        #self.image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path)[:7000] if img.endswith('.jpg')]
        self.label = index_array
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image_name = os.path.basename(image_path)  # 이미지 파일의 이름을 가져오기
        image = Image.open(image_path).convert('RGB')  # 이미지를 열고 RGB 형식으로 변환
        if self.transform:
            image = self.transform(image)
        return image,self.label

# 이미지 전처리를 위한 변환기 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조절
    transforms.ToTensor(),           # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])




In [ ]:
class CustomTrainDataset2(Dataset,Module):
    def __init__(self, folder_path, index_array, transform=None):
        self.folder_path = folder_path
        image_files=[os.path.join(folder_path,file) for file in os.listdir(folder_path)]
        self.image_paths=random.sample(image_files,3000)
        self.label = index_array
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image_name = os.path.basename(image_path)  # 이미지 파일의 이름을 가져오기
        image = Image.open(image_path).convert('RGB')  # 이미지를 열고 RGB 형식으로 변환
        if self.transform:
            image = self.transform(image)
        return image,self.label

# 이미지 전처리를 위한 변환기 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조절
    transforms.ToTensor(),           # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])


In [ ]:
class CustomValidDataset(Dataset,Module):
    def __init__(self, folder_path, index_array, transform=None):
        self.folder_path = folder_path
        image_files=[os.path.join(folder_path,file) for file in os.listdir(folder_path)]
        self.image_paths=random.sample(image_files,1000)
        #self.image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path)[:1000] if img.endswith('.jpg')]
        self.label = index_array
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image_name = os.path.basename(image_path)  # 이미지 파일의 이름을 가져오기
        image = Image.open(image_path).convert('RGB')  # 이미지를 열고 RGB 형식으로 변환
        if self.transform:
            image = self.transform(image)
        return image,self.label

# 이미지 전처리를 위한 변환기 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조절
    transforms.ToTensor(),           # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])


In [ ]:
# DataLoader를 사용하여 배치로 나누기
batch_size = 128

In [ ]:
import pickle

def save_data_loader(dataloader, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(dataloader, file)

def load_data_loader(file_path):
    with open(file_path, 'rb') as file:
        dataloader = pickle.load(file)
    return dataloader

In [ ]:
# DataLoader 객체를 파일에 저장
#save_dataset(datasets, 'dataset.pkl')

# 파일에서 DataLoader 객체 불러오기
train_loader = load_data_loader('/content/drive/MyDrive/종설/train_Dataloader.pkl')
valid_loader = load_data_loader('/content/drive/MyDrive/종설/valid_Dataloader.pkl')

In [ ]:
class ModifiedVGG19(nn.Module):
    def __init__(self, num_classes, input_size):
        super(ModifiedVGG19, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((16, 16))  # 512 크기의 이미지에 대해 Adaptive Pooling을 사용
        self.classifier = nn.Sequential(
            nn.Linear(512 * 16 * 16, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [ ]:
def conv2_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

In [ ]:
def conv3_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

In [ ]:
class VGG(nn.Module):
  def __init__(self, base_dim, num_classes):
    super(VGG,self).__init__()
    self.feature = nn.Sequential(
        conv2_block(3,base_dim),
        conv2_block(base_dim,2*base_dim),
        conv3_block(2*base_dim,4*base_dim),
        conv3_block(4*base_dim,8*base_dim),
        conv3_block(8*base_dim,8*base_dim),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(8*base_dim*7*7, 4096),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(4096,1000),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(1000,num_classes),
    )
  def forward(self,x):
    x = self.feature(x)
    x = x.view(x.size(0),-1)
    x = self.fc_layer(x)
    return x

In [ ]:
#VGG 클래스 인스턴스화
model = VGG(base_dim=64,num_classes=6)

#손실함수 및 최적화함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 모델 학습
num_epochs = 10
# 에폭을 반복하면서 학습과 검증을 수행
for epoch in range(num_epochs):
    '''
    checkpoint = torch.load('/content/drive/MyDrive/model_checkpoint_epoch{}.pt'.format(epoch))
    # 모델과 옵티마이저의 상태 복원
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    '''
    i=0
    # 학습 단계
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        print(f'epoch: {epoch}, batch: {i}')
        i+=1

    # 각 에폭 종료 후 검증 단계
    model.eval()  # 모델을 평가 모드로 전환
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for val_inputs, val_targets in valid_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_targets).item()

            # 정확도 계산
            _, predicted = torch.max(val_outputs, 1)
            correct_predictions += (predicted == val_targets).sum().item()
            total_samples += val_targets.size(0)

    average_val_loss = val_loss / len(valid_loader)
    accuracy = (correct_predictions / total_samples) * 100

    # 에폭 종료 후 출력
    print(f'Epoch [{epoch}/{num_epochs}], Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.2f}%')




    checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'Accuracy': "{:.2f}".format(accuracy)
    }

    # 저장 경로 설정
    save_path = '/content/drive/My Drive/VGG_checkpoint_epoch{}.pt'.format(epoch)

    # 딕셔너리를 파일로 저장
    torch.save(checkpoint, save_path)

    model.train()  # 모델을 학습 모드로 전환


epoch: 0, batch: 0
epoch: 0, batch: 1
epoch: 0, batch: 2
epoch: 0, batch: 3
epoch: 0, batch: 4
epoch: 0, batch: 5
epoch: 0, batch: 6
epoch: 0, batch: 7
epoch: 0, batch: 8
epoch: 0, batch: 9


KeyboardInterrupt: ignored